# Concept Network Tool

## Import data

Here user has to define the following:

- `DATA_PATH`: the folder including data files
- `DATA_FILES` : automatically detected `csv` files from `DATA_PATH` 


In [1]:
# Load R packages
pacman::p_load(tidyverse, udpipe)

In [2]:
DATA_PATH <- "processed_data"
DATA_FILES <- list.files(DATA_PATH, pattern = ".csv",full.names = TRUE,)

cat("Available datasets in", DATA_PATH, "\n",
    
    paste(paste(1:length(DATA_FILES) ,"\t", DATA_FILES, "\n"), sep = "\n"), "\n")


df_path <- DATA_FILES[as.numeric(readline(prompt="Select data: "))]
df <- read.csv(df_path)
cat(df_path, "selected.")

Available datasets in processed_data 
 1 	 processed_data/hs2014_processed.csv 
 
Select data: 1
processed_data/hs2014_processed.csv selected.

## Defining question and its concepts

### Interactively selected concepts

In [3]:
question <- readline(prompt="What is the question that was asked from the respondents?\n")
input_concepts <- readline(prompt="What are the concepts in this question? Separate multiple concepts with a comma.\n")
concepts <- trimws(stringr::str_split(input_concepts, ",", simplify = TRUE), "both")

cat("Following concepts selected:\n",
   paste(paste(1:length(concepts) ,"\t", concepts, "\n"), sep = "\n"), "\n")

What is the question that was asked from the respondents?
Miltä yksinäisyys tuntuu?
What are the concepts in this question? Separate multiple concepts with a comma.
yksinäisyys, yksinäisyyden tunne
Following concepts selected:
 1 	 yksinäisyys 
 2 	 yksinäisyyden tunne 
 


### Assigned variables

In [4]:
question = "Miltä yksinäisyys tuntuu? Yksinäisyyteen liittyy usein voimakkaita tunteita, jotka ovat erilaisia eri elämäntilanteissa. Jos haluat, voit kertoa yksinäisyyskokemuksiasi tässä"
concepts = c("yksinäisyys", "yksinäisyyden tunne")

cat("Following concepts selected:\n",
   paste(paste(1:length(concepts) ,"\t", concepts, "\n"), sep = "\n"), "\n")

Following concepts selected:
 1 	 yksinäisyys 
 2 	 yksinäisyyden tunne 
 


# Document Term Matrix

In [5]:
vocab = df %>% 
    filter(!xpos %in% c("Punct", "Symb"))%>% 
    pull(lemma) %>%
    unique()
cat(length(vocab), "words in vocabulary.")

term_frequencies <- document_term_frequencies(df %>% select(doc_id, lemma))
dtm = document_term_matrix(term_frequencies, vocabulary = vocab)

7734 words in vocabulary.

# Word frequencies for concept words

In [7]:
concept_counts <- function(dtm, concepts) {
    concept_list <- str_split(concepts, " ")
    
    output <- lapply(concept_list, function(i) {
        word_counts <- lapply(i, function(j){ifelse(j %in% colnames(dtm), sum(dtm[,j]) , NA)})
        df_words <- data.frame(term = i, count = unlist(word_counts))
        
    })
    
    names(output) <- concepts
    
    return(output)
}

concept_counts(dtm = dtm, concepts = concepts)

term,count
<chr>,<dbl>
yksinäisyys,1501
term,count
<chr>,<dbl>
yksinäisyyden,NA
tunne,674


# Co-occurences 

In [80]:
get_cooccurences <- function(data, concepts) {
    
    concept_list <- unlist(str_split(concepts, " "))
    
    cleaned_data <- data %>% 
    filter(!xpos %in% c("Punct", "Symb"))
    
    counts <- cooccurrence(cleaned_data, group = "doc_id", term = "lemma")
    
    concept_counts <- counts[counts$term1 %in% concept_list | counts$term2 %in% concept_list,]
    
    return(concept_counts)
}

get_cooccurences(data = df, concepts = concepts)

,term1,term2,cooc
,<chr>,<chr>,<dbl>
1,tuntua,yksinäisyys,1090
2,ihminen,yksinäisyys,944
3,tunne,yksinäisyys,849
4,yksin,yksinäisyys,781
6,yksinäisyys,ystävä,712
7,oma,yksinäisyys,693
8,muu,yksinäisyys,675
9,kokea,yksinäisyys,670
10,voida,yksinäisyys,642


# Context words for concept words

In [15]:
search_context_words <- function(data, concepts, size = 5) {
    concept_list <- str_split(concepts, " ")    
    
    cleaned_data <- data %>% 
    filter(!xpos %in% c("Punct", "Symb"))%>% 
    group_by(doc_id, paragraph_id, sentence_id) %>% 
    mutate(token_id = row_number(sentence_id))
    
    concept_words <- cleaned_data %>%
    filter(lemma %in% unlist(concept_list) | token %in% unlist(concept_list))
    
    
    get_window <- function(token_id, size) {

        indices <- lapply(token_id, function(x){
            start <- ifelse(x - size >= 1, x - size, 0)
            end <- x + size

            return(start:end)
        })
        return(indices)
    }
    
    concept_locations <- concept_words %>%
    tibble() %>%
    select(lemma, doc_id, sentence_id, token_id) %>%
    mutate(window = get_window(token_id, size = size)) %>%
    unnest_longer(window) %>%
    rename(concept_location = token_id, token_id = window, concept = lemma)
    
    context_windows <- concept_locations %>% 
    inner_join(cleaned_data, by = c("doc_id", "sentence_id", "token_id")) 
    
    
    return(context_windows)
}

context_windows <- search_context_words(data = df, concepts = concepts)

In [14]:
context_words_counts <- context_windows %>% 
    group_by(concept) %>% 
    count(lemma, sort = TRUE) %>%
    group_split()
context_words_counts

concept,lemma,n
<chr>,<chr>,<int>
tunne,tunne,726
tunne,yksinäisyys,226
tunne,tulla,64
tunne,että#ei,62
tunne,ulko#puolisuus,56
tunne,voida,52
tunne,kukaan,45
tunne,aiheuttaa,38
tunne,ihminen,38
